In [2]:
import pandas as pd 

# Read data
df = pd.read_excel('ETtoday_news.xlsx')
df

,category,title,link,date,content
0,生活,三仙台迎曙光 周邊實施彈性交通管制及嚴格取締酒駕,https://www.ettoday.net/news/20171230/1083085.htm,2017/12/30,▲台東縣三仙台遊憩區舉辦「2018三仙台迎元旦曙光活動」，元旦3天連續假期，預估將有龐大旅遊...
1,社會,酒駕被抓「先開直播」帶風向！嗆警惡霸強迫酒測 慘遭打臉,https://www.ettoday.net/news/20171230/1083126.htm,2017/12/30,▲盧男酒駕還嗆警方：拿出證據啊。（圖／翻攝自警方密錄器畫面）\n社會中心／綜合報導\n酒測車...
2,大陸,實力坑爹！老爸酒駕說謊 誠實兒氣炸秒伸「食指」：爸爸開的,https://www.ettoday.net/news/20180101/1083754.htm,2018/01/01,我們想讓你知道…老爸不認醉駕，兒子抱住親一口：是你開的！\n\n\n\n\n\n\n▲兒子認...
3,社會,酒後推「引擎發動」機車就算酒駕！ 他5年內被抓2次駕照飛了,https://www.ettoday.net/news/20180101/1083723.htm,2018/01/01,▲吳男酒後推行「引擎發動」的機車，被依酒駕開罰。（示意圖／資料照）\n社會中心／綜合報導\n...
4,大陸,悍女雙手交叉拒酒測！ 「站著尿一攤」給警察看...最後仍GG,https://www.ettoday.net/news/20180101/1083911.htm,2018/01/01,▲悍女雙手交叉胸前，兇悍地站在原地站著尿。（圖／翻攝自深圳交警微博）\n大陸中心／綜合報導\...
...,...,...,...,...,...
6325,體育,伍茲車禍當下其實「失去意識」 洛杉磯郡警局報告首度揭露,https://www.ettoday.net/news/20210306/1932284.htm,2021/03/06,▲老虎伍茲發生嚴重車禍意外。（圖／路透）\n\n記者路皓惟／綜合報導\n根據《美聯社》記者達...
6326,社會,台中小貨車「鬼切左轉」遭對向轎車鏟翻 駕駛命危,https://www.ettoday.net/news/20210306/1932237.htm,2021/03/06,記者李忠憲／台中報導\n台中西屯區發生嚴重車禍！一輛小貨車昨夜行經西屯區安和路，疑似轉彎時未...
6327,社會,驚悚影片曝！宜蘭休旅車衝撞三貼機車 2小孩「飛1F高」腿骨折送醫,https://www.ettoday.net/news/20210306/1932319.htm,2021/03/06,記者游芳男、陳韻如／宜蘭報導\n宜蘭礁溪5日晚間發生一起嚴重車禍！一輛休旅車行經中山路二段時...
6328,社會,車下掙扎畫面曝！高雄聯結車狠咬機車！7歲童骨盆以下骨折母女3重傷,https://www.ettoday.net/news/20210306/1932352.htm,2021/03/06,記者吳世龍、黃子倩／高雄報導\n高雄市阿蓮區今天上午11點10分在忠孝路和和平路的交叉路口發...


In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import re

In [5]:
# From page 783
def getETtodayNews(first_page, last_page):
    ETtoday_news = []
    count = 0
      
    for page in range(first_page, last_page-1, -1): 
        
        # 爬關鍵字搜尋頁面
        ETtoday_url = "https://www.ettoday.net/news_search/doSearch.php?keywords=%E9%85%92%E9%A7%95&idx=1&page="
        ETtoday_url += str(page)
        ETtoday_response = requests.get(ETtoday_url, timeout=(10, 15))
        ETtoday_soup = BeautifulSoup(ETtoday_response.text, 'html.parser')
        
        # 爬關鍵字搜尋頁面每篇新聞
        for i in range(-1, -11, -1):
            div = ETtoday_soup.find_all(class_="box_2")[i]
            content = ''

             # 取得「標題」
            title = div.find('a').text.strip()

            # 取得「分類」
            category = div.find(class_='date').find('a').text.strip()

             # 取得「日期」
            time = div.find(class_='date').text
            date = re.search(re.compile('20\d\d-\d\d-\d\d'),time).group(0).replace('-', '/')

            # 取得「網址」
            url = div.find('a')['href'] 

            # 登入該篇網址
            response = requests.get(url, timeout=(10, 15))
            soup = BeautifulSoup(response.text, 'html.parser')

            # 取得「內文」
            for s in soup.find_all(class_="story"):
                content += s.text.strip() 

            # 儲存資料
            ETtoday_news.append({'category': category, 'title': title, 'link': url, 'date': date, 'content': content})

            count += 1
        print("第" + str(page) + "頁爬取完成！")

    print("第{}頁到第{}頁，共{}筆資料".format(first_page, last_page, count))
    return (ETtoday_news)

In [12]:
df = getETtodayNews(832, 818)
df = pd.DataFrame(df)

第832頁爬取完成！
第831頁爬取完成！
第830頁爬取完成！
第829頁爬取完成！
第828頁爬取完成！
第827頁爬取完成！
第826頁爬取完成！
第825頁爬取完成！
第824頁爬取完成！
第823頁爬取完成！
第822頁爬取完成！
第821頁爬取完成！
第820頁爬取完成！
第819頁爬取完成！
第818頁爬取完成！
第832頁到第818頁，共150筆資料


In [14]:
df

,category,title,link,date,content
0,社會,國道躲死劫1年後翻車…女大生「目睹母被撞死」！忍痛辦後事,https://www.ettoday.net/news/20171130/1063458.htm,2017/11/30,▲國道翻車！女大生「目睹母被撞死」。（圖／記者吳欣晏翻攝）\n\n社會中心／彰化報導\n50...
1,生活,1201今日頭條／排放硫氧化物1公斤 空污費比茶葉蛋便宜,https://www.ettoday.net/news/20171201/1063627.htm,2017/12/01,【五報頭條】\n《ETtoday》\n吸到飽！排放硫氧化物1公斤 空污費竟「比茶葉蛋便宜」\...
2,國際,喝完啤酒就嘿咻 貴族學校女師床戰學生5次…落網流產,https://www.ettoday.net/news/20171201/1063625.htm,2017/12/01,國際中心／綜合報導\n澳洲雪梨一名女老師去年被發現和17歲的學生發生性行為，且至少一共嘿咻過...
3,社會,拒絕酒測「低消」改罰18萬 朱學恒酸：裝睡連一毛錢都不罰,https://www.ettoday.net/news/20171201/1063639.htm,2017/12/01,▲未來拒絕酒測、攔檢不停車的罰鍰，將比酒駕者的處罰還重。（圖／記者林世文攝，下同）\n社會中...
4,政治,酒駕重懲可望年底三讀 拒測罰18萬、同車連坐罰！,https://www.ettoday.net/news/20171201/1063797.htm,2017/12/01,▲酒駕重懲可望年底三讀。（示意圖／記者林世文攝）\n政治中心／綜合報導\n為遏止酒駕，立法院...
...,...,...,...,...,...
145,社會,快訊／騎車土城夜衝日月潭！37歲男疑打瞌睡 逆撞貨車爆頭亡,https://www.ettoday.net/news/20171229/1082688.htm,2017/12/29,圖文／鏡週刊\n \n最新消息！37歲的朱姓男子29日凌晨與友人從新北市出發，夜騎前往南投日...
146,體育,阪神看上旅美韓將 吳昇桓、朴炳鎬列調查名單,https://www.ettoday.net/news/20171230/1082933.htm,2017/12/30,▲吳昇桓。（圖／達志影像／美聯社） \n記者陳立勳／綜合報導\n日職阪神虎隊今年簽下台灣...
147,社會,逃逸外勞「騎車沒戴安全帽」 警取締意外發現安非他命,https://www.ettoday.net/news/20171230/1083066.htm,2017/12/30,▲台南市警永康分局永康派出所，查獲逃逸外勞持有安毒。（圖／記者林悅翻攝）\n記者林悅／南市報...
148,生活,三仙台迎曙光 周邊實施彈性交通管制及嚴格取締酒駕,https://www.ettoday.net/news/20171230/1083085.htm,2017/12/30,▲台東縣三仙台遊憩區舉辦「2018三仙台迎元旦曙光活動」，元旦3天連續假期，預估將有龐大旅遊...


In [16]:
df1 = df[(df["date"] > "2017/11/30)") & (df["date"] < "2018/01/01")].reset_index(drop=True)
df1

,category,title,link,date,content
0,生活,1201今日頭條／排放硫氧化物1公斤 空污費比茶葉蛋便宜,https://www.ettoday.net/news/20171201/1063627.htm,2017/12/01,【五報頭條】\n《ETtoday》\n吸到飽！排放硫氧化物1公斤 空污費竟「比茶葉蛋便宜」\...
1,國際,喝完啤酒就嘿咻 貴族學校女師床戰學生5次…落網流產,https://www.ettoday.net/news/20171201/1063625.htm,2017/12/01,國際中心／綜合報導\n澳洲雪梨一名女老師去年被發現和17歲的學生發生性行為，且至少一共嘿咻過...
2,社會,拒絕酒測「低消」改罰18萬 朱學恒酸：裝睡連一毛錢都不罰,https://www.ettoday.net/news/20171201/1063639.htm,2017/12/01,▲未來拒絕酒測、攔檢不停車的罰鍰，將比酒駕者的處罰還重。（圖／記者林世文攝，下同）\n社會中...
3,政治,酒駕重懲可望年底三讀 拒測罰18萬、同車連坐罰！,https://www.ettoday.net/news/20171201/1063797.htm,2017/12/01,▲酒駕重懲可望年底三讀。（示意圖／記者林世文攝）\n政治中心／綜合報導\n為遏止酒駕，立法院...
4,社會,酒駕男違規超車撞左轉車受傷 嗆聲：不賠錢就上法院！,https://www.ettoday.net/news/20171201/1063853.htm,2017/12/01,▲男子酒後騎車又違規超車不慎撞上左轉轎車。（圖／中壢交通分隊提供）\n記者楊熾興／桃園報導\...
...,...,...,...,...,...
144,社會,快訊／騎車土城夜衝日月潭！37歲男疑打瞌睡 逆撞貨車爆頭亡,https://www.ettoday.net/news/20171229/1082688.htm,2017/12/29,圖文／鏡週刊\n \n最新消息！37歲的朱姓男子29日凌晨與友人從新北市出發，夜騎前往南投日...
145,體育,阪神看上旅美韓將 吳昇桓、朴炳鎬列調查名單,https://www.ettoday.net/news/20171230/1082933.htm,2017/12/30,▲吳昇桓。（圖／達志影像／美聯社） \n記者陳立勳／綜合報導\n日職阪神虎隊今年簽下台灣...
146,社會,逃逸外勞「騎車沒戴安全帽」 警取締意外發現安非他命,https://www.ettoday.net/news/20171230/1083066.htm,2017/12/30,▲台南市警永康分局永康派出所，查獲逃逸外勞持有安毒。（圖／記者林悅翻攝）\n記者林悅／南市報...
147,生活,三仙台迎曙光 周邊實施彈性交通管制及嚴格取締酒駕,https://www.ettoday.net/news/20171230/1083085.htm,2017/12/30,▲台東縣三仙台遊憩區舉辦「2018三仙台迎元旦曙光活動」，元旦3天連續假期，預估將有龐大旅遊...


In [17]:
# Filter month
import jieba

df1['日期'] = df1['date'].apply(lambda x:x[:7])
df1

,category,title,link,date,content,日期
0,生活,1201今日頭條／排放硫氧化物1公斤 空污費比茶葉蛋便宜,https://www.ettoday.net/news/20171201/1063627.htm,2017/12/01,【五報頭條】\n《ETtoday》\n吸到飽！排放硫氧化物1公斤 空污費竟「比茶葉蛋便宜」\...,2017/12
1,國際,喝完啤酒就嘿咻 貴族學校女師床戰學生5次…落網流產,https://www.ettoday.net/news/20171201/1063625.htm,2017/12/01,國際中心／綜合報導\n澳洲雪梨一名女老師去年被發現和17歲的學生發生性行為，且至少一共嘿咻過...,2017/12
2,社會,拒絕酒測「低消」改罰18萬 朱學恒酸：裝睡連一毛錢都不罰,https://www.ettoday.net/news/20171201/1063639.htm,2017/12/01,▲未來拒絕酒測、攔檢不停車的罰鍰，將比酒駕者的處罰還重。（圖／記者林世文攝，下同）\n社會中...,2017/12
3,政治,酒駕重懲可望年底三讀 拒測罰18萬、同車連坐罰！,https://www.ettoday.net/news/20171201/1063797.htm,2017/12/01,▲酒駕重懲可望年底三讀。（示意圖／記者林世文攝）\n政治中心／綜合報導\n為遏止酒駕，立法院...,2017/12
4,社會,酒駕男違規超車撞左轉車受傷 嗆聲：不賠錢就上法院！,https://www.ettoday.net/news/20171201/1063853.htm,2017/12/01,▲男子酒後騎車又違規超車不慎撞上左轉轎車。（圖／中壢交通分隊提供）\n記者楊熾興／桃園報導\...,2017/12
...,...,...,...,...,...,...
144,社會,快訊／騎車土城夜衝日月潭！37歲男疑打瞌睡 逆撞貨車爆頭亡,https://www.ettoday.net/news/20171229/1082688.htm,2017/12/29,圖文／鏡週刊\n \n最新消息！37歲的朱姓男子29日凌晨與友人從新北市出發，夜騎前往南投日...,2017/12
145,體育,阪神看上旅美韓將 吳昇桓、朴炳鎬列調查名單,https://www.ettoday.net/news/20171230/1082933.htm,2017/12/30,▲吳昇桓。（圖／達志影像／美聯社） \n記者陳立勳／綜合報導\n日職阪神虎隊今年簽下台灣...,2017/12
146,社會,逃逸外勞「騎車沒戴安全帽」 警取締意外發現安非他命,https://www.ettoday.net/news/20171230/1083066.htm,2017/12/30,▲台南市警永康分局永康派出所，查獲逃逸外勞持有安毒。（圖／記者林悅翻攝）\n記者林悅／南市報...,2017/12
147,生活,三仙台迎曙光 周邊實施彈性交通管制及嚴格取締酒駕,https://www.ettoday.net/news/20171230/1083085.htm,2017/12/30,▲台東縣三仙台遊憩區舉辦「2018三仙台迎元旦曙光活動」，元旦3天連續假期，預估將有龐大旅遊...,2017/12


In [18]:
# Filter category
df = df[df.category.isin(["國際", "地方", "大陸", "影劇", "政治" , "法律", "生活", "社會", "體育"])].reset_index(drop=True)

df

,category,title,link,date,content
0,社會,國道躲死劫1年後翻車…女大生「目睹母被撞死」！忍痛辦後事,https://www.ettoday.net/news/20171130/1063458.htm,2017/11/30,▲國道翻車！女大生「目睹母被撞死」。（圖／記者吳欣晏翻攝）\n\n社會中心／彰化報導\n50...
1,生活,1201今日頭條／排放硫氧化物1公斤 空污費比茶葉蛋便宜,https://www.ettoday.net/news/20171201/1063627.htm,2017/12/01,【五報頭條】\n《ETtoday》\n吸到飽！排放硫氧化物1公斤 空污費竟「比茶葉蛋便宜」\...
2,國際,喝完啤酒就嘿咻 貴族學校女師床戰學生5次…落網流產,https://www.ettoday.net/news/20171201/1063625.htm,2017/12/01,國際中心／綜合報導\n澳洲雪梨一名女老師去年被發現和17歲的學生發生性行為，且至少一共嘿咻過...
3,社會,拒絕酒測「低消」改罰18萬 朱學恒酸：裝睡連一毛錢都不罰,https://www.ettoday.net/news/20171201/1063639.htm,2017/12/01,▲未來拒絕酒測、攔檢不停車的罰鍰，將比酒駕者的處罰還重。（圖／記者林世文攝，下同）\n社會中...
4,政治,酒駕重懲可望年底三讀 拒測罰18萬、同車連坐罰！,https://www.ettoday.net/news/20171201/1063797.htm,2017/12/01,▲酒駕重懲可望年底三讀。（示意圖／記者林世文攝）\n政治中心／綜合報導\n為遏止酒駕，立法院...
...,...,...,...,...,...
133,社會,快訊／騎車土城夜衝日月潭！37歲男疑打瞌睡 逆撞貨車爆頭亡,https://www.ettoday.net/news/20171229/1082688.htm,2017/12/29,圖文／鏡週刊\n \n最新消息！37歲的朱姓男子29日凌晨與友人從新北市出發，夜騎前往南投日...
134,體育,阪神看上旅美韓將 吳昇桓、朴炳鎬列調查名單,https://www.ettoday.net/news/20171230/1082933.htm,2017/12/30,▲吳昇桓。（圖／達志影像／美聯社） \n記者陳立勳／綜合報導\n日職阪神虎隊今年簽下台灣...
135,社會,逃逸外勞「騎車沒戴安全帽」 警取締意外發現安非他命,https://www.ettoday.net/news/20171230/1083066.htm,2017/12/30,▲台南市警永康分局永康派出所，查獲逃逸外勞持有安毒。（圖／記者林悅翻攝）\n記者林悅／南市報...
136,生活,三仙台迎曙光 周邊實施彈性交通管制及嚴格取締酒駕,https://www.ettoday.net/news/20171230/1083085.htm,2017/12/30,▲台東縣三仙台遊憩區舉辦「2018三仙台迎元旦曙光活動」，元旦3天連續假期，預估將有龐大旅遊...
